# Reproduce Branin-function example Results from Perdikaris with QUEENS

The purpose of this notebook is the reproduction of some of the results presented in [Perdikaris et al. 2017](http://rspa.royalsocietypublishing.org/content/473/2198/20160751). The code published along with the original publication can be found [here](https://github.com/paraklas/NARGP).

### Input File

In [1]:
# Define multi-fidelity model
options = { 
    "output_dir" : "/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output",
    "experiment-name" : "perdikaris_nargp_branin_test",
    "database"  :{
      "address"   : "localhost:27017"
    },
    "method": {
      "method_name": "monte_carlo",
      "method_options" :{
        "seed" : 44,
        "num_samples" : 500,
        "model" : "mf_gp_icm_surrogate_model"
      }
    },
    "mf_gp_icm_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
    "approximation_interface" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression"
    },
    "mf_gp_regression" : {
      "type" : "mf_icm_gp_approximation_gpy",
    },
    "mf_gp_nargp_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface2",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
    "approximation_interface2" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression2"
    },
    "mf_gp_regression2" : {
      "type" : "mf_nar_gp_approximation_gpy_3_levels",
    },
    "thruth_model_iterator" : {
      "method_name": "lhs_mf",
      "method_options" :{
        "seed" : 42,
        "num_samples" : [80, 40, 10],
        "num_iterations" :10,
        "mode" : "nested"
      }
    },
    "truth_model" :{
      "type" : "multi_fidelity_model",
      "model_hierarchy" : ["lofi_branin", "medfi_branin", "hifi_branin"],
      "eval_cost_per_level" :[1, 1, 1],
      "parameters" : "parameters"
    },
    "hifi_branin" : {
      "type" : "simulation_model",
      "interface" : "interface_hifi"
    },
    "medfi_branin" : {
      "type" : "simulation_model",
      "interface" : "interface_medfi"
    },
    "lofi_branin" : {
      "type" : "simulation_model",
      "interface" : "interface_lofi"
    },
    "interface_lofi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/branin_lofi.py"
    },
    "interface_hifi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/branin_hifi.py"
    },
    "interface_medfi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/branin_medfi.py"
    },
    "parameters" : {
        "x1" : {
            "type" : "FLOAT",
            "size" : 1,
            #"min"  : -5,
            #"max"  : 10,
            "distribution" : "uniform",
            "distribution_parameter" : [-5,10]
        },
        "x2" : {
            "type" : "FLOAT",
            "size" : 1,
            #"min"  : 0,
            #"max"  : 15,
            "distribution" : "uniform",
            "distribution_parameter" : [0,15]
        }
    }
}



### Run QUEENS

In [2]:
#from pqueens.iterators.iterator import Iterator
#build iterator
#my_iterator = Iterator.from_config_create_iterator(options)
# perform analysis
#my_iterator.run()

### Build step by step

In [3]:
from pqueens.models.model import Model
import numpy as np

np.random.seed(options["method"]["method_options"]["seed"])
num_samples = options["method"]["method_options"]["num_samples"]


# create surrogate models 
icm_model = Model.from_config_create_model("mf_gp_icm_surrogate_model", options)
nargp_model = Model.from_config_create_model("mf_gp_nargp_surrogate_model", options)

distribution_info = nargp_model.get_parameter_distribution_info()
numparams = len(distribution_info)
samples = np.zeros((num_samples, numparams))

i = 0
for distribution in distribution_info:
    # get appropriate random number generator
    random_number_generator = getattr(np.random, distribution['distribution'])
    # make a copy
    my_args = list(distribution['distribution_parameter'])
    my_args.extend([num_samples])
    samples [:, i] = random_number_generator(*my_args)
    i += 1
        


/Applications/anaconda/envs/test/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Compute Results

In [4]:
from pqueens.example_simulator_functions.branin_hifi import branin_hifi
from pqueens.example_simulator_functions.branin_lofi import branin_lofi
from pqueens.example_simulator_functions.branin_medfi import branin_medfi

# generate meshgrid for reference plots
x1_grid = np.arange(-5, 10, 0.5)
x2_grid = np.arange(0,15, 0.5)
samples2 = np.meshgrid(x1_grid, x2_grid)
samples2_1 = np.asarray(samples2[0].ravel()).reshape(-1,1)
samples2_2 = np.asarray(samples2[1].ravel()).reshape(-1,1)
samples2_arr = np.hstack((samples2_1,samples2_2))

# update model
nargp_model.update_model_from_sample_batch(samples2_arr)

ref_outputs_hifi = branin_hifi(samples2[0],samples2[1])
ref_outputs_medfi = branin_medfi(samples2[0],samples2[1])
ref_outputs_lofi = branin_lofi(samples2[0],samples2[1])

nargp_outputs = nargp_model.evaluate()
nargp_samples_lofi, nargp_samples_medfi, nargp_samples_hifi = nargp_model.subordinate_iterator.samples
nargp_results_lofi, nargp_results_medfi, nargp_results_hifi = nargp_model.subordinate_iterator.outputs



Size of inputs in LHS(80, 2)
Inputs [[ 4.38834787 13.20908354]
 [ 5.55288148  7.28559185]
 [-4.66281965  5.63263665]
 [-0.58452     1.22674587]
 [ 0.97289886  2.74451811]
 [ 8.11573884  0.62587409]
 [ 6.83183573 13.64107022]
 [ 2.51323164  2.10296644]
 [ 3.05129317  2.53712899]
 [-0.84566805  7.98290141]
 [ 1.27967798  8.35230315]
 [ 1.40747535 13.8355534 ]
 [ 8.36747702 14.76778324]
 [-2.55280968  3.38971057]
 [ 9.33278697  6.7584945 ]
 [ 8.89210114 10.06813822]
 [ 6.06071971 12.57961625]
 [-1.66340259  6.27627596]
 [ 4.01889588 10.46315267]
 [ 7.73225912 14.41644453]
 [ 6.13225339 11.424517  ]
 [ 8.24346948  8.18311654]
 [-3.10117615 12.37703206]
 [ 4.72091412  9.56686349]
 [-0.98582132  4.17192054]
 [-2.05135787 14.8314605 ]
 [-1.09325373  7.05484873]
 [-2.27363091  7.34305021]
 [ 0.44019878  0.52421113]
 [ 7.80987131 12.15496077]
 [-2.79312621  9.00341659]
 [ 8.83592107  7.79733292]
 [ 5.82824027 10.289526  ]
 [ 1.08890759 12.35541061]
 [ 6.59845239  3.96999269]
 [ 9.07329227 10.53

reconstraining parameters GP_regression.Gaussian_noise.variance


Optimization restart 1/30, f = 113.51469180045255
Optimization restart 2/30, f = -69.15039930026455
Optimization restart 3/30, f = -69.1503993041521
Optimization restart 4/30, f = -69.15039930445477
Optimization restart 5/30, f = -69.15039929680265
Optimization restart 6/30, f = -69.15039930440916
Optimization restart 7/30, f = -69.15039929443077
Optimization restart 8/30, f = -69.15039930111391
Optimization restart 9/30, f = -69.15039929357059
Optimization restart 10/30, f = -69.15039930420104
Optimization restart 11/30, f = -69.15039930441172
Optimization restart 12/30, f = -69.15039930226197
Optimization restart 13/30, f = -69.15039930440875
Optimization restart 14/30, f = -69.15039930425631
Optimization restart 15/30, f = -69.15039930440959
Optimization restart 16/30, f = -69.15039930429785
Optimization restart 17/30, f = -69.15039930448535
Optimization restart 18/30, f = -69.1503993044519
Optimization restart 19/30, f = -69.15039930445187
Optimization restart 20/30, f = -69.150399

reconstraining parameters GP_regression.Gaussian_noise.variance



Optimization restart 29/30, f = -69.15039930443152
Optimization restart 30/30, f = -69.15039930443808
Optimization restart 1/30, f = -20.78388602331328
Optimization restart 2/30, f = -20.783855378923242
Optimization restart 3/30, f = -20.78387603990177
Optimization restart 4/30, f = -20.783879289827844
Optimization restart 5/30, f = -20.783885075804143
Optimization restart 6/30, f = -20.78388126280238
Optimization restart 7/30, f = -28.578888494593457
Optimization restart 8/30, f = -20.783861812937744
Optimization restart 9/30, f = -20.783875416838093
Optimization restart 10/30, f = -22.187918672532813
Optimization restart 11/30, f = -15.891368053021129
Optimization restart 12/30, f = -20.78387501909463
Optimization restart 13/30, f = -20.783877034745508
Optimization restart 14/30, f = -20.783883247536814
Optimization restart 15/30, f = -20.783867750542335
Optimization restart 16/30, f = -15.891395425913952
Optimization restart 17/30, f = -28.578893823532738
Optimization restart 18/30

reconstraining parameters GP_regression.Gaussian_noise.variance


Optimization restart 1/30, f = 14.189357111565055
Optimization restart 2/30, f = 11.215520427656404
Optimization restart 3/30, f = 12.251444579911528
Optimization restart 4/30, f = 12.251444090528452
Optimization restart 5/30, f = 12.251444078119984
Optimization restart 6/30, f = 11.21552045124499
Optimization restart 7/30, f = 12.251444075958045
Optimization restart 8/30, f = 12.251444081332076
Optimization restart 9/30, f = 12.251444286178657
Optimization restart 10/30, f = 12.25144407950702
Optimization restart 11/30, f = 13.106210901433798
Optimization restart 12/30, f = 12.251444680916666
Optimization restart 13/30, f = 12.251444084679608
Optimization restart 14/30, f = 12.25144412153946
Optimization restart 15/30, f = 12.251444235603831
Optimization restart 16/30, f = 11.215520410171713


 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:136: RuntimeWarning:overflow encountered in square
 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:invalid value encountered in add


Optimization restart 17/30, f = 12.25287760569744
Optimization restart 18/30, f = 12.25144407097343
Optimization restart 19/30, f = 14.189385332047875
Optimization restart 20/30, f = 11.215520410455369
Optimization restart 21/30, f = 13.154496459790574
Optimization restart 22/30, f = 12.251447261440847
Optimization restart 23/30, f = 12.251444543316357
Optimization restart 24/30, f = 13.103868927006026
Optimization restart 25/30, f = 11.215525008729045
Optimization restart 26/30, f = 12.251444115570429
Optimization restart 27/30, f = 13.10621045034778
Optimization restart 28/30, f = 11.215520403981214
Optimization restart 29/30, f = 11.215520424107904
Optimization restart 30/30, f = 12.251444209684317


### Plot results

In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go
import numpy as np
plotly.offline.init_notebook_mode(connected=True)

colorscale1 = [[0, 'rgb(0,0,255)'], [1, 'rgb(0,0,255)']]
colorscale2 = [[0, 'rgb(0,255,0)'], [1, 'rgb(0,255,0)']]
colorscale3 = [[0, 'rgb(255,0,0)'], [1, 'rgb(255,0,0)']]

marker=dict(color='rgb(127, 127, 127)',size=2, symbol='circle', line=dict(color='rgb(204, 204, 204)',width=1),opacity=0.9)

hifi_data = go.Surface(x=samples2[0].tolist(), y=samples2[1].tolist(), z=ref_outputs_hifi, opacity=1,colorscale=colorscale1,showscale=False)
medfi_data = go.Surface(x=samples2[0].tolist(), y=samples2[1].tolist(), z=ref_outputs_medfi, opacity=0.9,colorscale=colorscale2)
lofi_data = go.Surface(x=samples2[0].tolist(), y=samples2[1].tolist(), z=ref_outputs_lofi, opacity=0.9, colorscale=colorscale3)

lofi_train_data = go.Scatter3d(x=nargp_samples_lofi[:,0].tolist(), y=nargp_samples_lofi[:,1].tolist(),z=nargp_results_lofi.flatten(),mode='markers',marker=marker)
medfi_train_data = go.Scatter3d(x=nargp_samples_medfi[:,0].tolist(), y=nargp_samples_medfi[:,1].tolist(),z=nargp_results_medfi.flatten(),mode='markers',marker=marker)
hifi_train_data = go.Scatter3d(x=nargp_samples_hifi[:,0].tolist(), y=nargp_samples_hifi[:,1].tolist(),z=nargp_results_hifi.flatten(),mode='markers',marker=marker)

#hifi_predictions = go.Scatter3d(x=samples[:,0].tolist(), y=samples[:,1].tolist(),z=nargp_outputs.flatten(),mode='markers',marker=marker)
hifi_predictions = go.Surface(x=samples2[0].tolist(), y=samples2[1].tolist(), z=nargp_outputs.reshape(30,-1), opacity=1, colorscale=colorscale2,showscale=False)

#data = [hifi_data, medfi_data, lofi_data, lofi_train_data, medfi_train_data, hifi_train_data]
data = [hifi_predictions, hifi_data]

layout = go.Layout(
    title='Branin Function',
    autosize=True,
    width=500,
    height=500,
    showlegend = False,
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='branin-function')
    